In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2021-12-21 01:19:21.478834: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-21 01:19:21.478877: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.5.0
2.5.2


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
EPOCHS = 15
#YEAR = '2000'
YEAR = '2010'

df = pd.read_csv('../data/census/census_%s.csv' % YEAR)
df.dropna(subset=['name'], inplace=True)
df.replace('(S)', 0, inplace=True)
df

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2442977,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4
1,JOHNSON,2,1932812,655.24,1483.42,58.97,34.63,0.54,0.94,2.56,2.36
2,WILLIAMS,3,1625252,550.97,2034.39,45.75,47.68,0.46,0.82,2.81,2.49
3,BROWN,4,1437026,487.16,2521.56,57.95,35.6,0.51,0.87,2.55,2.52
4,JONES,5,1425470,483.24,3004.80,55.19,38.48,0.44,1,2.61,2.29
...,...,...,...,...,...,...,...,...,...,...,...
162249,DIETZMANN,160975,100,0.03,90062.93,96,0,0,0,0,0
162250,DOKAS,160975,100,0.03,90062.96,94,0,0,0,0,0
162251,DONLEA,160975,100,0.03,90062.99,94,0,0,0,0,6
162252,DORIOTT,160975,100,0.03,90063.03,89,0,0,0,5,0


## Resampling with weight

In [3]:
sdf = df.sample(1000000, weights=df['count'], replace=True)

## Assign race by pertcentage

In [4]:
from numpy.random import choice

races = ['white', 'black', 'api', 'hispanic']

def to_race(c):
    w = np.array(c).astype(float)
    if w.sum() == 0:
        return 'white'
    probs = w/w.sum()
    return choice(races, p=probs)

sdf['race'] = sdf[['pctwhite', 'pctblack', 'pctapi', 'pcthispanic']].apply(lambda c: to_race(c), axis=1)
sdf

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic,race
2,WILLIAMS,3,1625252,550.97,2034.39,45.75,47.68,0.46,0.82,2.81,2.49,white
16,TAYLOR,17,751209,254.67,6886.93,65.38,28.42,0.56,0.76,2.41,2.46,black
2,WILLIAMS,3,1625252,550.97,2034.39,45.75,47.68,0.46,0.82,2.81,2.49,white
42423,KASA,42378,512,0.17,81468.90,74.02,8.4,10.35,0,0,5.47,white
8516,LEPAGE,8517,3865,1.31,66581.46,94.1,0.44,0.6,0.26,1.97,2.64,white
...,...,...,...,...,...,...,...,...,...,...,...,...
860,SMALL,861,40212,13.63,39207.27,62.66,30.81,0.58,1.41,1.93,2.61,white
30,WALKER,31,523129,177.34,9862.30,58.66,35.05,0.45,0.87,2.55,2.41,black
91,ALVAREZ,92,233983,79.32,16697.86,5.18,0.6,1.16,0.38,0.23,92.45,hispanic
9396,KEMPTON,9397,3460,1.17,67672.28,92.54,1.79,0.55,0.46,1.97,2.69,hispanic


## Check the correctness of race assignment

In [5]:
df[df.name == 'SMITH']

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2442977,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4


In [6]:
xdf = sdf[sdf.name=='SMITH'].groupby(['race']).agg({'name': 'count'})
xdf * 100 / xdf.sum()

,name
race,
api,0.464946
black,24.054815
hispanic,2.691790
white,72.788450


In [7]:
# Additional features
sdf['name_last'] = sdf.name.str.title()
sdf.groupby('race').agg({'name_last': 'count'})

,name_last
race,
api,49841
black,123962
hispanic,165010
white,661187


In [8]:
len(sdf)

1000000

## Preprocessing the input data

In [9]:
# only last name in Census data
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
#vect = CountVectorizer(analyzer='char', ngram_range=(2, 2), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_
len(vocab)

976

In [10]:
import operator
sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))
cols = list(map(operator.itemgetter(0), sorted_vocab))

In [11]:
count_df = pd.DataFrame(a.todense(), columns=cols)
count_df

,N,O,Aa,Ab,Ac,Ad,Ae,Af,Ag,Ah,...,zp,zq,zr,zs,zt,zu,zv,zw,zy,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
count_df.sum().sort_values(ascending=False).describe()

count       976.000000
mean       6322.976434
std       18148.435773
min           3.000000
25%          75.750000
50%         845.500000
75%        4990.750000
max      254005.000000
dtype: float64

In [13]:
pd.set_option('display.max_rows', 20)
count_df.sum().sort_values(ascending=False)

er    254005
ll    174191
es    143601
am    131473
he    123684
       ...  
Iu         3
tq         3
Ej         3
vj         3
mv         3
Length: 976, dtype: int64

In [14]:
# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = ['UNK']
words_list.extend([w[1] for w in words])
num_words = len(words_list)
print("num_words = %d" % num_words)

num_words = 977


In [15]:
def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

Max feature len = 14, Avg. feature len = 6


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [16]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


In [17]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 32)            31264     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 114,212
Trainable params: 114,212
Non-trainable params: 0
_________________________________________________________________
None


2021-12-21 01:25:35.989612: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-21 01:25:35.989652: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-21 01:25:35.989669: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-server-2): /proc/driver/nvidia/version does not exist
2021-12-21 01:25:35.989888: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15


2021-12-21 01:25:36.307378: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-21 01:25:36.307923: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2250000000 Hz


22500/22500 [==============================] - 445s 19ms/step - loss: 0.6870 - accuracy: 0.7653 - val_loss: 0.6236 - val_accuracy: 0.7864
Epoch 2/15
22500/22500 [==============================] - 415s 18ms/step - loss: 0.6177 - accuracy: 0.7884 - val_loss: 0.6052 - val_accuracy: 0.7917
Epoch 3/15
22500/22500 [==============================] - 413s 18ms/step - loss: 0.6026 - accuracy: 0.7931 - val_loss: 0.5953 - val_accuracy: 0.7946
Epoch 4/15
22500/22500 [==============================] - 412s 18ms/step - loss: 0.5930 - accuracy: 0.7959 - val_loss: 0.5899 - val_accuracy: 0.7968
Epoch 5/15
22500/22500 [==============================] - 412s 18ms/step - loss: 0.5862 - accuracy: 0.7980 - val_loss: 0.5860 - val_accuracy: 0.7978
Epoch 6/15
22500/22500 [==============================] - 412s 18ms/step - loss: 0.5829 - accuracy: 0.7984 - val_loss: 0.5822 - val_accuracy: 0.7980
Epoch 7/15
22500/22500 [==============================] - 411s 18ms/step - loss: 0.5836 - accuracy: 0.7980 - val_loss

## Confusion Matrix

In [19]:
p = model.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

6250/6250 - 27s
              precision    recall  f1-score   support

         api       0.83      0.57      0.68      9968
       black       0.55      0.08      0.14     24792
    hispanic       0.88      0.77      0.82     33002
       white       0.79      0.96      0.87    132238

    accuracy                           0.80    200000
   macro avg       0.76      0.60      0.63    200000
weighted avg       0.78      0.80      0.76    200000

[[  5708     36    773   3451]
 [   271   1946    251  22324]
 [   157    120  25321   7404]
 [   731   1441   2544 127522]]


## Save model

In [20]:
model.save('./census/lstm/census%s_ln_lstm.h5' % YEAR)

In [21]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./census/lstm/census%s_ln_vocab.csv' % YEAR, index=False, encoding='utf-8')

In [22]:
y_pred = model.predict_classes(X_train, verbose=2)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_train, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_train, axis=1), y_pred))

/home/suriyant_gmail_com/.conda/envs/jupyter-env/lib/python3.8/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


25000/25000 - 105s
              precision    recall  f1-score   support

         api       0.85      0.58      0.69     39873
       black       0.57      0.08      0.14     99170
    hispanic       0.88      0.77      0.82    132008
       white       0.79      0.97      0.87    528949

    accuracy                           0.81    800000
   macro avg       0.77      0.60      0.63    800000
weighted avg       0.78      0.81      0.76    800000

[[ 23294    135   3146  13298]
 [   890   8066    918  89296]
 [   604    390 101828  29186]
 [  2666   5496   9882 510905]]


In [23]:
xdf.to_csv('./census/lstm/census%s_race.csv' % YEAR, columns=[])